### Create the environment
To do so, we select an API endpoint and collect a token from our configuration. <br>

In [ ]:
eosc_perf_api="https://performance.services.fedcloud.eu/api/v1"

### Configure the arguments and terms for your search

#### Search for the benchmark id that produced our result
You can get a list of all available benchmarks using GET /benchmarks

In [ ]:
benchmarks=$(curl -X 'GET' "$eosc_perf_api/benchmarks?docker_image=deephdc/deep-oc-benchmarks_cnn&docker_tag=benchmark")
benchmark=$(echo $benchmarks | jq '.items[0]')
echo $benchmark | jq '.json_schema = "..."'

In [ ]:
benchmark_id=$(echo $benchmark | jq -r '.id')
curl -X 'GET' "$eosc_perf_api/results?benchmark_id=$benchmark_id" \
  -H 'accept: application/json' | jq '.items'

#### Search for the site id used to run our benchmark
You can get a list of all available benchmarks using GET /sites

In [ ]:
sites=$(curl -X 'GET' "$eosc_perf_api/sites?name=ForHLR2")
site=$(echo $sites | jq '.items[0]')
echo $site | jq

In [ ]:
site_id=$(echo $site | jq -r '.id')
curl -X 'GET' "$eosc_perf_api/results?site_id=$site_id" \
  -H 'accept: application/json' | jq '.items'

#### Search for the flavor id used to run our benchmark
If needed you can also get a list of all available flavors in that site using GET /sites{id}/flavors

In [ ]:
site_id=$(echo $site | jq -r '.id')
flavors=$(curl -X 'GET' "$eosc_perf_api/sites/$site_id/flavors:search?terms=haicore-gpu8")
flavor=$(echo $flavors | jq '.items[0]')
echo $flavor | jq

In [ ]:
flavor_id=$(echo $flavor | jq -r '.id')
curl -X 'GET' "$eosc_perf_api/results?flavor_id=$site_id" \
  -H 'accept: application/json' | jq '.items'

#### Search for the tags to relate your result
Collect the tags you want to link to your result so users can find it easily. <br>
> If you do not know the name of the tag you can use :search as generic filter.

In [ ]:
tag_gpu=$(curl -X 'GET' "$eosc_perf_api/tags?name=gpu" | jq '.items[0]')
echo $tag_gpu | jq
tag_hpc=$(curl -X 'GET' "$eosc_perf_api/tags?name=hpc" | jq '.items[0]')
echo $tag_hpc | jq

In [ ]:
tag_1=$(echo $tag_gpu | jq -r '.id')
tag_2=$(echo $tag_hpc | jq -r '.id')
curl -X 'GET' "$eosc_perf_api/results?tag_id=$tag_1&tag_id=$tag_2" \
  -H 'accept: application/json' | jq '.items'

#### Search for results between dates
If is also possible to filter results by upload and execution date.

In [1]:
upload_before='2021-10-16'
execution_after='2021-10-14'

curl -X 'GET' "$eosc_perf_api/results?upload_before=$upload_before&execution_after=$execution_after" \
  -H 'accept: application/json' | jq '.items'

curl: (3) URL using bad/illegal format or missing URL


#### Search using custom filters
If is also possible to filter results specific values inside the result (if you know the result structure).
> If you do not want results that does not include the field in the response, we recommend that you use a benchmark_id

In [ ]:
benchmark_id='5a0f5641-01ae-4f48-b5d1-84cd9c87dfac'
filter_1='machine_config.gpu_info.count%20%3E%204'
filter_2='machine_config.gpu_info.count%20%3C%2020'

curl -X 'GET' "$eosc_perf_api/results?benchmark_id=$benchmark_id&filters=$filter_1&filters=$filter_2" \
  -H 'accept: application/json' | jq '.items'

#### Configure sorting and use pagination
For all the previous options, it is possible to sort the results using the following fields:
- "id": Result id.
- "upload_datetime": EOSC Performance upload datetime of the result.
- "json": Result json values.
- "execution_datetime": Execution date of the benchmark.
- "benchmark_id": Benchmark id used to obtain the result.
- "flavor_id": Favor id used to run the benchmark.
- "site_id": Site id where the benchmark was executed.

In addition, sometimes you might get more results that expected. In such case you will have to use pagination to collect all the items. To do so you can use the following parameters:
- per_page: The number of items to be displayed on a page (maximum 100)
- page: The return page number (1 indexed)

In [ ]:
curl -X 'GET' "$eosc_perf_api/results?per_page=4&page=2&sort_by=%2Bid" \
  -H 'accept: application/json' | jq '.items'